In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 57.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00


In [4]:
from gliner import GLiNER

def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['label'] == current['label'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['text'] = text[current['start']: next_entity['end']].strip()
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    # Append the last entity
    merged.append(current)
    return merged


# model = GLiNER.from_pretrained("numind/NuNerZero")
model = GLiNER.from_pretrained("numind/NuZero_token")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

NuZero_token_token_metrics.txt:   0%|          | 0.00/961 [00:00<?, ?B/s]

zero_shot_performance_unzero_token.png:   0%|          | 0.00/43.1k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
labels = ["location","date","person","event", "company", "organization", "position"]
labels = [l.lower() for l in labels]

text = """Fiat has completed its buyout of Chrysler, making the U.S. business a wholly-owned subsidiary of the Italian
carmaker as it gears up to use their combined resources to turn around its loss-making operations in
Europe. The company announced on January 1 that it had struck a $4.35 billion deal - cheaper than analysts
had expected - to gain full control of Chrysler, ending more than a year of tense talks that had obstructed Chief Executive Sergio Marchionne's efforts to create the
world's seventh-largest auto maker."""

entities = model.predict_entities(text, labels, threshold=0.4)

entities = merge_entities(entities)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Fiat => organization
Chrysler => company
U.S. => location
Italian => location
Europe => location
January 1 => date
Chrysler => company
Chief Executive => position
Sergio Marchionne => person


In [6]:
!pip install wikipedia

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=a72701d090cba19b819481d31a71b087933f5173b098448d16f1e2fd60686d3e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [8]:
!pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requ

In [7]:
import wikipedia
from tqdm import tqdm

In [10]:
!pip install -U langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.3 MB/s eta 0:00:00a 0:00:01


In [14]:
!pip install tiktoken

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.2 MB/s eta 0:00:0000:0100:010m


In [12]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter

In [17]:
raw_documents = WikipediaLoader(query="Large Language Model").load()

# Define chunking strategy
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=20
)
# Chunk the document
documents = text_splitter.split_documents(raw_documents)
for d in tqdm(documents):
    del d.metadata["summary"]

100%|██████████| 29/29 [00:00<00:00, 179402.38it/s]


In [18]:
for doc in documents:
    print(doc.metadata['source'])

https://en.wikipedia.org/wiki/Large_language_model
https://en.wikipedia.org/wiki/Language_model
https://en.wikipedia.org/wiki/Language_model
https://en.wikipedia.org/wiki/Language_model
https://en.wikipedia.org/wiki/Claude_(language_model)
https://en.wikipedia.org/wiki/Llama_(language_model)
https://en.wikipedia.org/wiki/BLOOM_(language_model)
https://en.wikipedia.org/wiki/T5_(language_model)
https://en.wikipedia.org/wiki/T5_(language_model)
https://en.wikipedia.org/wiki/BERT_(language_model)
https://en.wikipedia.org/wiki/Gemini_(language_model)
https://en.wikipedia.org/wiki/Generative_pre-trained_transformer
https://en.wikipedia.org/wiki/Chinchilla_(language_model)
https://en.wikipedia.org/wiki/Prompt_engineering
https://en.wikipedia.org/wiki/Foundation_model
https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)
https://en.wikipedia.org/wiki/GPT-3
https://en.wikipedia.org/wiki/Mistral_AI
https://en.wikipedia.org/wiki/PaLM
https://en.wikipedia.org/wiki/Modeling_languag

In [19]:
# Step 1: Use a set to track seen sources
unique_documents = []
seen_sources = set()

# Step 2: Iterate through the documents and store only unique sources
for doc in documents:
    source = doc.metadata['source']
    if source not in seen_sources:
        unique_documents.append(doc)
        seen_sources.add(source)

# Step 3: Iterate through unique documents and print their sources
for doc in unique_documents:
    print(doc.metadata['source'])

https://en.wikipedia.org/wiki/Large_language_model
https://en.wikipedia.org/wiki/Language_model
https://en.wikipedia.org/wiki/Claude_(language_model)
https://en.wikipedia.org/wiki/Llama_(language_model)
https://en.wikipedia.org/wiki/BLOOM_(language_model)
https://en.wikipedia.org/wiki/T5_(language_model)
https://en.wikipedia.org/wiki/BERT_(language_model)
https://en.wikipedia.org/wiki/Gemini_(language_model)
https://en.wikipedia.org/wiki/Generative_pre-trained_transformer
https://en.wikipedia.org/wiki/Chinchilla_(language_model)
https://en.wikipedia.org/wiki/Prompt_engineering
https://en.wikipedia.org/wiki/Foundation_model
https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)
https://en.wikipedia.org/wiki/GPT-3
https://en.wikipedia.org/wiki/Mistral_AI
https://en.wikipedia.org/wiki/PaLM
https://en.wikipedia.org/wiki/Modeling_language
https://en.wikipedia.org/wiki/Perplexity_AI
https://en.wikipedia.org/wiki/Model_collapse
https://en.wikipedia.org/wiki/GPT-4o
https://en.w

In [21]:
labels = [
    "model",            # Large Language Models (e.g., GPT-4, BERT, T5, LLaMA)
    "organization",     # Companies or institutions (e.g., OpenAI, Google)
    "person",           # Key figures in AI/ML (e.g., Sam Altman, Geoffrey Hinton)
    "event",            # Conferences, events, or milestones (e.g., NeurIPS, ACL)
    "dataset",          # Datasets used to train models (e.g., Common Crawl)
    "metric",           # Performance evaluation metrics (e.g., perplexity, accuracy)
    "technology",       # Techniques and methods (e.g., Transformer, attention mechanism)
    "application",      # Applications of LLMs (e.g., ChatGPT, translation, summarization)
    "company",          # AI companies (e.g., DeepMind, Microsoft)
    "position",         # Job titles or roles (e.g., CEO, researcher)
    "location",         # Geographical locations (e.g., cities, countries)
    "date",             # Important dates or time references
    "product"           # AI products (e.g., cloud platforms, AI tools)
]

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [26]:
# Define the text splitter with chunking strategy
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    separators=["\n\n", "\n"]
)

chunks_entities = []
entity_list = []
duplicates = set()

for doc in tqdm(unique_documents):
    print(f"The document is: {doc.metadata['source']}\n")
    chunks = text_splitter.split_text(doc.page_content)
    for text in chunks:
        entities = model.predict_entities(text, labels, threshold=0.7)
        entities = merge_entities(entities)
        
        chunk_entities = set() 
        for entity in entities:
            chunk_entities.add(entity["text"])
            
            if entity["text"] not in duplicates:
                duplicates.add(entity["text"])
                entity_list.append((entity["text"], "=>", entity["label"]))
        
        chunks_entities.append(list(chunk_entities))

  0%|          | 0/25 [00:00<?, ?it/s]

The document is: https://en.wikipedia.org/wiki/Large_language_model



  4%|▍         | 1/25 [00:10<04:06, 10.27s/it]

The document is: https://en.wikipedia.org/wiki/Language_model



  8%|▊         | 2/25 [00:13<02:23,  6.24s/it]

...

 92%|█████████▏| 23/25 [03:37<00:18,  9.36s/it]

The document is: https://en.wikipedia.org/wiki/Open-source_artificial_intelligence



 96%|█████████▌| 24/25 [03:42<00:08,  8.11s/it]

The document is: https://en.wikipedia.org/wiki/Retrieval-augmented_generation



100%|██████████| 25/25 [03:58<00:00,  9.53s/it]


In [27]:
chunks_entities[:2]

[['LLM', 'language models', 'large language model'],
 ['human language corpora',
  'August 2024',
  'transformer-based architecture',
  'artificial neural networks',
  'data',
  'models']]

In [28]:
chunks[9]

"\nThe model feeds this relevant retrieved information into the LLM via prompt engineering of the user's original query. Newer implementations (as of 2023) can also incorporate specific augmentation modules with abilities such as expanding queries into multiple domains, and using memory and self-improvement to learn from previous retrievals."

In [29]:
entity_list[:4]

[('large language model', '=>', 'model'),
 ('LLM', '=>', 'model'),
 ('language models', '=>', 'model'),
 ('August 2024', '=>', 'date')]

In [31]:
models = []
orgs = []
persons = []
events = []
datasets = []
metrics = []
techs = []
apps = []
comps = []
poses = []
locs = []
dates = []
prods = []

for e in tqdm(entity_list):
    s,p, o = e
    if o == 'model':
        models.append(s.lower())
    elif o == 'organization':
        orgs.append(s.lower())
    elif o == 'person':
        persons.append(s.lower())
    elif o == 'event':
        events.append(s.lower())
    elif o == 'dataset':
        datasets.append(s.lower())
    elif o == 'metric':
        metrics.append(s.lower())
    elif o == 'technology':
        techs.append(s.lower())
    elif o == 'application':
        apps.append(s.lower())
    elif o == 'company':
        poses.append(s.lower())
    elif o == 'position':
        orgs.append(s.lower())
    elif o == 'location':
        locs.append(s.lower())
    elif o == 'date':
        dates.append(s.lower())
    elif o == 'product':
        prods.append(s.lower())

100%|██████████| 612/612 [00:00<00:00, 623491.39it/s]


In [32]:
len(models), len(orgs), len(persons), len(events), len(datasets), len(metrics), len(techs), len(apps), len(comps), len(poses), len(locs), len(dates), len(prods) 

(167, 59, 45, 16, 57, 27, 99, 23, 0, 20, 19, 57, 23)

In [35]:
!pip install litellm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 55.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.1 MB/s eta 0:00:00


In [36]:
from litellm import completion
from typing import List
import json

In [38]:
def format_entities(ent_list:List[str]) -> str:
    return "\n\n".join([e for e in ent_list])

print(format_entities(chunks_entities[9]))

2024

Transformer architecture

recurrent neural network variants

Mamba


In [ ]:
system_message = """Extract all the relationships between the following entities ONLY based on the given context. 
Return a list of JSON objects. For example:

<Examples>
    [{{"subject": "John", "relationship": "lives in", "object": "US"}},
    {{"subject": "Eifel towel", "relationship": "is located in", "object": "Paris"}},
    {{"subject": "Hayao Miyazaki", "relationship": "is", "object": "Japanese animator"}}]
</Examples>

- ONLY return triples and nothing else. None of 'subject', 'relationship' and 'object' can be empty.

Entities: \n\n{entities}

"""
# example for a particular set of entities
i = 3

ents = format_entities(chunks_entities[i])
text = chunks[i]

user_message = "Context: {text}\n\nTriples:"
response = completion(
  # api_key="" 
  model="gpt-3.5-turbo",
  messages=[{"content": system_message.format(entities=ents),"role": "system"}, {"content": user_message.format(text=text),"role": "user"}],
  max_tokens=1000,
  format = "json"
)

triples = json.loads(response.choices[0].message.content)
triples